In [ ]:
global_features = ['Scr', 'Temp']  # глобальные признаки, которые влияют на все участки
local_features = ['kh_eff', 'h_all', 'ntg', 'hgvk', 'minLgwc',
                  'Vdren', 'Sdren', 'minL2', 'minL3', 'kHmin1', 'kHmin2', 'kHmin3'] # локальные признаки, которые влияют только на свой участок


In [ ]:
def normalize_by_group(df, group_col, columns_to_norm):
    df_norm = df.copy()
    for col in columns_to_norm:
        df_norm[col] = df.groupby(group_col)[col].transform(
            lambda x: (x - x.mean()) / x.std(ddof=0)
        )
    return df_norm


In [ ]:
X_full = df[global_features + local_features].copy()

# Нормализуем локальные признаки
X_local_norm = normalize_by_group(df, group_col='Temp', columns_to_norm=local_features)

# Объединяем нормализованные локальные + глобальные
X_final = pd.concat([X_local_norm[local_features], df[global_features]], axis=1)


In [ ]:
y = df['q_i']
y = df['OFR']


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=300, max_depth=10, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [ ]:
print("R2:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("MAE:", mean_absolute_error(y_test, y_pred))
from scipy.stats import spearmanr
print("Spearman:", spearmanr(y_test, y_pred).correlation)
